## Comparison of Pre-Trained Models

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

In [0]:
# import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Download Data

In [0]:
!wget  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
!unzip -q concrete_data_week4.zip

--2020-04-13 15:45:51--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  32.8MB/s    in 7.9s    

2020-04-13 15:46:00 (31.6 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [0]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [0]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)
# dataset_dir = ""
# image_generator = data_generator.flow_from_directory(
#     dataset_dir,
#     batch_size=4,
#     class_mode='categorical',
#     seed=24
#     )
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [0]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [0]:
model = Sequential()

In [0]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
model.add(Dense(num_classes, activation='softmax'))

In [0]:
model.layers

In [0]:
model.layers[0].trainable = False

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [0]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
301/301 [==============================] - 116s 384ms/step - loss: 0.1369 - accuracy: 0.9463 - val_loss: 0.0318 - val_accuracy: 0.9935
Epoch 2/2
301/301 [==============================] - 114s 380ms/step - loss: 0.0249 - accuracy: 0.9944 - val_loss: 0.0187 - val_accuracy: 0.9954


In [0]:
model.save('classifier_vgg_model.h5')

In [0]:
ls

classifier_resnet_model.h5  concrete_data_week4/     __MACOSX/
classifier_vgg_model.h5     concrete_data_week4.zip  sample_data/


## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


In [0]:
from tensorflow.keras.models import load_model

import os
import numpy as np
import matplotlib.pyplot as plt

model_resnet = load_model('classifier_resnet_model.h5')

In [0]:
model_resnet.save('classifier_resnet1_model.h5')

In [0]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    shuffle=False,
    class_mode='categorical')
len(test_generator)

Found 500 images belonging to 2 classes.


5

In [44]:
# VGG16 model
vgg_acc = model.evaluate_generator(test_generator, len(test_generator))
vgg_acc[1]*100

99.59999918937683

In [45]:
# Resnet50 model
res_acc = model_resnet.evaluate_generator(test_generator, len(test_generator))
res_acc[1]*100

100.0

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

In [0]:
# VGG16 model
vgg_pred = model.predict_generator(test_generator, len(test_generator))
vgg_pred

array([[1.14803679e-05, 9.99988556e-01],
       [1.54390356e-09, 1.00000000e+00],
       [5.67124953e-05, 9.99943256e-01],
       [6.93807015e-05, 9.99930620e-01],
       [3.05401130e-07, 9.99999642e-01],
       [3.01565450e-07, 9.99999642e-01],
       [1.97974936e-04, 9.99802053e-01],
       [3.74812480e-05, 9.99962568e-01],
       [2.43056638e-05, 9.99975681e-01],
       [3.95828858e-04, 9.99604166e-01],
       [3.31283081e-05, 9.99966860e-01],
       [9.17626676e-06, 9.99990821e-01],
       [2.85091932e-08, 1.00000000e+00],
       [2.44434807e-04, 9.99755561e-01],
       [1.35593553e-04, 9.99864340e-01],
       [1.63112093e-11, 1.00000000e+00],
       [5.37108281e-06, 9.99994636e-01],
       [2.43770210e-05, 9.99975681e-01],
       [7.20969729e-06, 9.99992847e-01],
       [1.75370616e-04, 9.99824703e-01],
       [1.25488325e-04, 9.99874473e-01],
       [3.08928847e-08, 1.00000000e+00],
       [1.77559647e-04, 9.99822438e-01],
       [6.11946161e-05, 9.99938846e-01],
       [6.098693

In [0]:
# Resnet50 model
res_pred = model_resnet.predict_generator(test_generator, len(test_generator))
res_pred

array([[1.21573430e-06, 9.99998808e-01],
       [9.73977194e-07, 9.99999046e-01],
       [2.18514765e-06, 9.99997854e-01],
       [2.22097515e-06, 9.99997735e-01],
       [4.12372901e-04, 9.99587595e-01],
       [8.33352516e-08, 9.99999881e-01],
       [5.35308245e-05, 9.99946475e-01],
       [1.56728311e-05, 9.99984384e-01],
       [3.46142187e-04, 9.99653816e-01],
       [1.26633386e-05, 9.99987364e-01],
       [2.65380280e-04, 9.99734581e-01],
       [2.55155810e-05, 9.99974489e-01],
       [8.93519682e-05, 9.99910593e-01],
       [9.75274088e-06, 9.99990225e-01],
       [5.98618317e-05, 9.99940157e-01],
       [1.61581222e-04, 9.99838352e-01],
       [9.66452353e-06, 9.99990344e-01],
       [5.67603893e-05, 9.99943256e-01],
       [2.55389787e-05, 9.99974489e-01],
       [1.92964828e-04, 9.99807060e-01],
       [1.68952611e-05, 9.99983072e-01],
       [2.00264587e-08, 1.00000000e+00],
       [4.80876915e-05, 9.99951959e-01],
       [9.44518659e-04, 9.99055445e-01],
       [9.765331

In [0]:
def model_labels(arr):
  if(arr[0]>arr[1]):
    return "Negative"
  else:
    return "Positive"

In [47]:
for i in range(5):
  print(model_labels(vgg_pred[i]))

Positive
Positive
Positive
Positive
Positive


In [48]:
for i in range(5):
  print(model_labels(res_pred[i]))

Positive
Positive
Positive
Positive
Positive
